In [ ]:
from pyroSAR import identify
name = 'data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip'
scene = identify(name)
print(scene)

In [ ]:
from pyroSAR.auxdata import dem_autoload
from spatialist import Vector

site = 'data/aoi/aoi_tmp/aoi_tmp.shp'
vrt = 'data/mosaic.vrt'

with Vector(site) as vec:
    vrt = dem_autoload(geometries=[vec],
                       demType='SRTM 1Sec HGT',
                       vrt=vrt,
                       buffer=0.1)


In [ ]:
from pyroSAR.snap import geocode
geocode(infile='data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip',
        outdir='data/out', test=True)

In [ ]:
from pyroSAR import Archive
dbfile = 'data/scenes.db'
with Archive(dbfile) as archive:
    archive.insert(scene)


In [ ]:
from spatialist import Vector
archive = Archive('data/scenes.db')
outdir = 'data/out'
maxdate = '20220202'
site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'
with Vector(site_shp) as site:
    selection_proc = archive.select(vectorobject=site,
                                    processdir=outdir,
                                    maxdate=maxdate,
                                    sensor=('S1A', 'S1B'),
                                    product='GRD',
                                    acquisition_mode='IW',
                                    vv=1)
archive.close()

In [ ]:
# test groupsize, see what grousize is needed to get all steps into one group; currently it's 8 steps + read/write == groupsize of 8
geocode(infile='data/download/S1A_IW_GRDH_1SDV_20210206T052511_20210206T052536_036465_0447F4_09CA.zip',
        outdir='data/out',
        spacing=10,
        shapefile=site_shp,
        polarizations=['VV', 'VH'],
        scaling='dB',
        cleanup=True,
        tmpdir='data/tmp',
        groupsize=8,
        test=True)

from pyroSAR.snap.auxil import groupbyWorkers
groupbyWorkers('data/out/S1A__IW___D_20210206T052511_Cal_NR_Orb_TF_TC_dB_proc.xml', n=8)

In [10]:
from pyroSAR.snap import geocode
import os


files = [os.path.join('data/download/', each) for each in os.listdir('data/download/') if each.endswith('.zip')]
# satfiles = [os.path.join(root, name)
#             for root, dirs, files in os.walk('../sen1ost/data/download/SAR/GRD/2021')
#             for name in files
#             if name.endswith("zip")]
# satfiles = satfiles[0:3]
site_shp = 'data/aoi/aoi_tmp/aoi_tmp.shp'

for single_file in files:
# for single_file in satfiles:
    geocode(infile=single_file,
        outdir='data/out/raw',
        spacing=10,
        shapefile=site_shp,
        polarizations=['VV', 'VH'],
        scaling='dB',
        cleanup=True,
        tmpdir='data/tmp',
        groupsize=8)



In [ ]:
import fiona
import rasterio
from rasterio import mask

# To mask the data
with fiona.open('data/aoi/aoi_tmp/aoi_tmp.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open("data/out/S1A__IW___D_20210206T052511_VH_gamma0-rtc_db.tif") as src:
    out_image, transformed = rasterio.mask.mask(src, shapes, crop=True, filled=True)
    out_profile = src.profile.copy()

out_profile.update({'width': out_image.shape[2],'height': out_image.shape[1], 'transform': transformed})
with rasterio.open("data/out/tehran.tif", 'w', **out_profile) as dst:
    dst.write(out_image)

In [26]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os

tif_files = [file for file in os.listdir('data/out/raw') if file.endswith(".tif")]
site_shp = 'data/aoi/flotow.shp'

# Open the .tif file
for file in tif_files:
    with rasterio.open(os.path.join('data/out/raw', file)) as src:
        # Open the .shp file using geopandas
        shapefile = gpd.read_file(site_shp)

        # Clip the .tif file using the .shp file geometry
        clipped, transform = mask(src, shapefile.geometry, crop=True)
        out_profile = src.profile.copy()
        out_profile.update({'width': clipped.shape[2],'height': clipped.shape[1], 'transform': transform})

        # Save the clipped .tif file
        with rasterio.open(os.path.join('data/out/cut', file), 'w', **out_profile) as dst:
            dst.write(clipped)


In [24]:
tif_files = [file for file in os.listdir('data/out/raw') if file.endswith(".tif")]
[os.path.join('data/out', file) for file in tif_files]
tif_files

['S1A__IW___D_20210206T052511_VH_gamma0-rtc_db.tif',
 'S1B__IW___A_20210203T170010_VH_gamma0-rtc_db.tif',
 'S1A__IW___A_20210209T170054_VV_gamma0-rtc_db.tif',
 'S1A__IW___A_20210209T170054_VH_gamma0-rtc_db.tif',
 'S1B__IW___A_20210203T170010_VV_gamma0-rtc_db.tif',
 'S1A__IW___D_20210206T052511_VV_gamma0-rtc_db.tif']

['S1A__IW___D_20210206T052511_VH_gamma0-rtc_db.tif',
 'S1B__IW___A_20210203T170010_VH_gamma0-rtc_db.tif',
 'S1A__IW___A_20210209T170054_VV_gamma0-rtc_db.tif',
 'S1A__IW___A_20210209T170054_VH_gamma0-rtc_db.tif',
 'S1B__IW___A_20210203T170010_VV_gamma0-rtc_db.tif',
 'S1A__IW___D_20210206T052511_VV_gamma0-rtc_db.tif']